<a href="https://colab.research.google.com/github/Kshitij-1008/NN-micrograd-and-makemore/blob/main/Makemore_Backprop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'karpathy-makemore-names-text:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5135485%2F8586183%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240621%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240621T165654Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D80238bf59301524f042933cf4df74ba313465356d34f0f897af32a6b846d325ff3a63c842d3d5dbb2a5715a2bbd0461385bcbd842536d99b01b985ca70dd239c45f7d2b5c19f252885226f27b158a297ece2845c3f705326b1ee04d5b508d488cc62b6efe8b5a84aed256270c8bb0655eec34f3145dd374dc05d73466e5b2650b82d58214d61668c7030fbdaac5fb5c02c93b08b3c5ae66c6d9df0d5c520631483c6c83accc8dbc807b9ace3ea5598ce8033052a2bea24c0295df737af681911f356debe883ee914651f7fee08ca5c985ab9e147fe22962d9e3e70fd7ebc70f69a4c78863cc4018ff31e31c5cf4393e623efebc0bc1d2959de76aeab26637c5d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/karpathy-makemore-names-text/names.txt


In [ ]:
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F

## Preparation of the dataset

In [ ]:
# normal qualities of the txt file contents
words = open(r'/kaggle/input/karpathy-makemore-names-text/names.txt', 'r').read().splitlines()
print(len(words))
print(words[:10])

32033
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [ ]:
# mappings from characters to integers and vice versa
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)
vocab_size = len(stoi)
print(vocab_size)


{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


### Building the dataset

In [ ]:
block_size = 3 #no. of characters to determine to next character in line.

def build_dataset(words):
    X,Y = [], []

    for w in words: #first eg: 'emma'
        context = [0]*block_size

        for char in w + '.': #first eg: 'e+m+m+a+.'
            idx = stoi[char]
            Y.append(idx)
            X.append(context)

            context = context[1:]+[idx]

    X= torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xval, Yval = build_dataset(words[n1:n2])
Xtest, Ytest = build_dataset(words[n2:])

Xtr[:5], Ytr[:5]

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


(tensor([[ 0,  0,  0],
         [ 0,  0, 25],
         [ 0, 25, 21],
         [25, 21,  8],
         [21,  8,  5]]),
 tensor([25, 21,  8,  5, 14]))

### Initializing the parameters

In [ ]:
n_embed = 10
n_hidden = 64

g = torch.Generator().manual_seed(2147483647) #for deterministic and reproducable results
C = torch.randn((vocab_size, n_embed),           generator=g)
#Layer 1
W1 = torch.randn((n_embed*block_size, n_hidden), generator=g) * (5/3)/((n_embed*block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
#Layer 2
W2 = torch.randn((n_hidden, vocab_size),         generator=g) * 0.1
b2 = torch.randn(vocab_size,                     generator=g) * 0.1
#BatchNorm parameters
bngain = torch.ones((1, n_hidden))*0.1+1.0
bnbias = torch.zeros((1, n_hidden))*0.1

# Note: initialization of many of these parameters has been done in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

4137


### Utility function

In [ ]:
# utility function to compare gradients calculated manually and using PyTorch
def cmp(s, dt, t):
    ex = torch.all(dt==t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt-t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [ ]:
batch_size = 32
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]
Xb[:5], Yb[:5]

(tensor([[ 1,  1,  4],
         [18, 14,  1],
         [11,  5,  9],
         [ 0,  0,  1],
         [12, 15, 14]]),
 tensor([ 8, 14, 15, 22,  0]))

In [ ]:
# forward pass, edited to be able to manually calculate backward pass

emb = C[Xb]
emb_cat = emb.view(emb.shape[0], -1) #reshape the tensor

#Layer 1
h_pre_bn = emb_cat @ W1 + b1 # pre activation and batch normalization

#BatchNorm Layer
bnmeani = (1/batch_size)*h_pre_bn.sum(0, keepdim=True)   # mean of the batch
bndiff = h_pre_bn - bnmeani                              # difference between batch and batch mean
bndiff2 = bndiff**2                                      # square of the difference between batch and mean for variance
bnvari = 1/(batch_size-1) * bndiff2.sum(0, keepdim=True) # variance
bnvar_inv = (bnvari+1e-5)**(-0.5)                        # epsilon = 1e-5
bnraw = bndiff*bnvar_inv
h_pre_act = bngain * bnraw + bnbias
#Non-Linearity
h = torch.tanh(h_pre_act)

#Layer 2
logits = h @ W2 + b2

#cross-entropy loss
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes  #for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(batch_size), Yb].mean()

#PyTorch backward pass
for p in parameters:
    p.grad = None

for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, norm_logits, logit_maxes, logits, h, h_pre_act, bnraw, bnvar_inv, bnvari, bndiff2, bndiff, h_pre_bn, bnmeani, emb_cat, emb]:
    if t.requires_grad == False:
        t.requires_grad = True
    t.retain_grad()

loss.backward()
loss

tensor(3.3482, grad_fn=<NegBackward0>)

In [ ]:
Xb[0,2], C[4], emb[0,2]

(tensor(4),
 tensor([-0.9648, -0.2321, -0.3476,  0.3324, -1.3263,  1.1224,  0.5964,  0.4585,
          0.0540, -1.7400], grad_fn=<SelectBackward0>),
 tensor([-0.9648, -0.2321, -0.3476,  0.3324, -1.3263,  1.1224,  0.5964,  0.4585,
          0.0540, -1.7400], grad_fn=<SelectBackward0>))

## Exercise 1: backprop through the whole thing manually, backpropagating through exactly all of the variables as they are defined in the forward pass above, one by one

In [ ]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(batch_size), Yb] = -1/batch_size

dprobs = (1/probs) * dlogprobs

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
dcounts = counts_sum_inv*dprobs

dcounts_sum = (-1*counts_sum**-2) * dcounts_sum_inv
dcounts +=  torch.ones_like(counts) * dcounts_sum

dnorm_logits = counts * dcounts
dlogit_maxes = (-1*dnorm_logits).sum(1, keepdim=True)
dlogits = 1 * dnorm_logits + F.one_hot(logits.max(1).indices, num_classes=vocab_size) * dlogit_maxes

dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)
dh_pre_act = (1.0-h**2) * dh

dbngain = (bnraw * dh_pre_act).sum(0, keepdim=True)
dbnbias = dh_pre_act.sum(0, keepdim=True)

dbnraw = bngain*dh_pre_act
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
dbnvari = -0.5 * ((bnvari+1e-5)**(-1.5)) * dbnvar_inv

dbndiff2 = 1/(batch_size-1) * torch.ones_like(bndiff2) * dbnvari
dbndiff = bnvar_inv * dbnraw + 2*bndiff * dbndiff2

dbnmeani = (-dbndiff).sum(0)
dh_pre_bn = dbndiff + (1/batch_size)*torch.ones_like(h_pre_bn)*dbnmeani


demb_cat = dh_pre_bn @ W1.T
dW1 = emb_cat.T @ dh_pre_bn
db1 = dh_pre_bn.sum(0)
demb = demb_cat.view(emb.shape)

dC = torch.zeros_like(C)
for i in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[i,j]
        dC[ix] += demb[i,j]


In [ ]:
# COMPARISON
cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('h_pre_act', dh_pre_act, h_pre_act)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvari', dbnvari, bnvari)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('h_pre_bn', dh_pre_bn, h_pre_bn)
cmp('emb_cat', demb_cat, emb_cat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
h_pre_act       | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10
bngain          | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnbias          | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
bnraw   

## Exercise 2: Backprop through cross_entropy in one expression

In [ ]:
# forward pass

# before:
"""
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes  #for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(batch_size), Yb].mean()
"""
# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.348198175430298 diff: 0.0


In [ ]:
# backward pass
# -----------------
# YOUR CODE HERE :)
dlogits = F.softmax(logits, 1)
dlogits[range(batch_size), Yb] -= 1
dlogits /= batch_size
# TODO. my solution is 3 lines
# -----------------

cmp('logits', dlogits, logits)

logits          | exact: False | approximate: True  | maxdiff: 8.614733815193176e-09


## Exercise 3: backprop through batchnorm but all in one go

In [ ]:
# forward pass

# before:
"""
bnmeani = (1/batch_size)*h_pre_bn.sum(0, keepdim=True)   # mean of the batch
bndiff = h_pre_bn - bnmeani                              # difference between batch and batch mean
bndiff2 = bndiff**2                                      # square of the difference between batch and mean for variance
bnvari = 1/(batch_size-1) * bndiff2.sum(0, keepdim=True) # variance
bnvar_inv = (bnvari+1e-5)**(-0.5)                        # epsilon = 1e-5
bnraw = bndiff*bnvar_inv
h_pre_act = bngain * bnraw + bnbias
"""
# now:
h_pre_act_fast = bngain * (h_pre_bn-h_pre_bn.mean(0, keepdim=True))/(h_pre_bn.var(0, keepdim=True)+1e-5)**0.5 + bnbias
print('max diff:', (h_pre_act_fast - h_pre_act).abs().max().item())

max diff: 4.76837158203125e-07


In [ ]:
# backward pass
n = batch_size
# before we had:
"""
dbngain = (bnraw * dh_pre_act).sum(0, keepdim=True)
dbnbias = dh_pre_act.sum(0, keepdim=True)

dbnraw = bngain*dh_pre_act
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
dbnvari = -0.5 * ((bnvari+1e-5)**(-1.5)) * dbnvar_inv

dbndiff2 = 1/(batch_size-1) * torch.ones_like(bndiff2) * dbnvari
dbndiff = bnvar_inv * dbnraw + 2*bndiff * dbndiff2

dbnmeani = (-dbndiff).sum(0)
dh_pre_bn = dbndiff + (1/batch_size)*torch.ones_like(h_pre_bn)*dbnmeani
"""

# -----------------
# YOUR CODE HERE :)
dh_pre_bn = bngain*bnvar_inv/n * (n*dh_pre_act - dh_pre_act.sum(0) - n/(n-1)*bnraw*(dh_pre_act*bnraw).sum(0))
# -----------------

cmp('hprebn', dh_pre_bn, h_pre_bn) # I can only get approximate to be true, my maxdiff is 9e-10

hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


## Exercise 4: Putting it all together

In [ ]:
# Training a MLP with our backward pass

n_embed = 10
n_hidden = 200

g = torch.Generator().manual_seed(2147483647) #for deterministic and reproducable results
C = torch.randn((vocab_size, n_embed),           generator=g)
#Layer 1
W1 = torch.randn((n_embed*block_size, n_hidden), generator=g) * (5/3)/((n_embed*block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1 #useless
#Layer 2
W2 = torch.randn((n_hidden, vocab_size),         generator=g) * 0.1
b2 = torch.randn(vocab_size,                     generator=g) * 0.1
#BatchNorm parameters
bngain = torch.ones((1, n_hidden))*0.1+1.0
bnbias = torch.zeros((1, n_hidden))*0.1
#Mean and std buffers, (not trainable)
bnmean_running = torch.zeros((1, n_hidden)) * 0.1 + 1
bnstd_running = torch.ones((1, n_hidden)) * 0.1


parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True


#optimization
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
with torch.no_grad():

    for i in range(max_steps):

    #     minibatch construct
        ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
        Xb, Yb = Xtr[ix], Ytr[ix]

    #     embedding
        emb = C[Xb]
        embcat = emb.view(emb.shape[0], -1)

        # ----- Linear Layer 1 -----
        # hidden layer pre-activation
        hprebn = embcat @ W1 + b1
        # ---- BatchNorm layer ----
        # mean of hidden layer pre-activation
        bnmeani = 1 / n * hprebn.sum(0, keepdim = True)
        # difference between pre-activation and mean
        bndiff = hprebn - bnmeani
        # square of difference
        bndiff2 = bndiff ** 2
        # variance of hidden layer pre-activation (Bessel's correction (n-1))
        bnvar = 1 / (n - 1) * (bndiff2).sum(0, keepdim = True)
        # get the std of hidden layer pre-activation, and invert it (remember: we need to divide by it)
        bnvar_inv = 1 / (bnvar + 1e-5) ** 0.5
        # batch norm layer output
        bnraw = bndiff * bnvar_inv
        # batch norm layer output with gain and bias (scale and shift)
        hpreact = bngain * bnraw + bnbias

        with torch.no_grad():
            bnmean_running = 0.99*bnmean_running * 0.01*bnmeani
            bnstd_running = 0.99*bnstd_running * 0.01*bnvar**0.5
        # activation function
        h = torch.tanh(hpreact)


        # ----- Linear Layer 2 -----
        # logits
        logits = h @ W2 + b2

        # ----- Cross Entropy Loss -----
        loss = F.cross_entropy(logits, Yb)

        # ----- Backpropagation and Zero_grad-----
        for p in parameters:
            p.grad = None
#         loss.backward()

        dlogits = F.softmax(logits, 1)
        dlogits[range(n), Yb] -= 1
        dlogits /= n
        # 2nd layer backprop
        dh = dlogits @ W2.T
        dW2 = h.T @ dlogits
        db2 = dlogits.sum(0)
        # tanh
        dhpreact = (1.0 - h**2) * dh
        # batchnorm backprop
        dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
        dbnbias = dhpreact.sum(0, keepdim=True)
        dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
        # 1st layer
        dembcat = dhprebn @ W1.T
        dW1 = embcat.T @ dhprebn
        db1 = dhprebn.sum(0)
        # embedding
        demb = dembcat.view(emb.shape)
        dC = torch.zeros_like(C)
        for k in range(Xb.shape[0]):
            for j in range(Xb.shape[1]):
                ix = Xb[k,j]
                dC[ix] += demb[k,j]
        grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]

        # update
        lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
        for p, grad in zip(parameters, grads):
          #p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
          p.data += -lr * grad # new way

        # track stats
        if i % 10000 == 0: # print every once in a while
            print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
        lossi.append(loss.log10().item())

#     if i >= 100: # TODO: delete early breaking when you're ready to train the full net
#         break

12297
      0/ 200000: 3.8279
  10000/ 200000: 2.1609
  20000/ 200000: 2.4227
  30000/ 200000: 2.4362
  40000/ 200000: 2.0088
  50000/ 200000: 2.4084
  60000/ 200000: 2.4508
  70000/ 200000: 2.1090
  80000/ 200000: 2.3592
  90000/ 200000: 2.2353
 100000/ 200000: 1.9750
 110000/ 200000: 2.3438
 120000/ 200000: 2.0156
 130000/ 200000: 2.4772
 140000/ 200000: 2.3107
 150000/ 200000: 2.1108
 160000/ 200000: 1.9497
 170000/ 200000: 1.8004
 180000/ 200000: 2.0284
 190000/ 200000: 1.8848


In [ ]:
# # useful for checking your gradients
# for p,g in zip(parameters, grads):
#     cmp(str(tuple(p.shape)), g, p)

In [ ]:
with torch.no_grad():
    emb = C[Xtr]
    embcat = emb.view(emb.shape[0],-1)
    hpreact = embcat @ W1 + b1

    bnmean = hpreact.mean(0, keepdim=True)
    bnvar = hpreact.var(0, keepdim=True, unbiased=True)

In [ ]:
@torch.no_grad()
def split_loss(split):
    x, y = {"train" : (Xtr, Ytr), "dev" : (Xval, Yval),"test": (Xtest, Ytest)}[split]
    emb = C[x] #shape: (No of examples/batch_size, block_size, embeddings)
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    hpreact = bnbias * (hpreact-bnmean) * (bnvar+1e-5)**-0.5 + bnbias
    h = torch.tanh(hpreact) #shape: (batch_size, no_hidden)
    logits = h @ W2 + b2 #shape: (batch_size, vocab_size)

    loss = F.cross_entropy(logits, y)
    print(f"{split} : {loss.item()}")

split_loss('train')
split_loss('dev')

train : 3.4889321327209473
dev : 3.4822936058044434


In [ ]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
    x,y = {'train': (Xtr, Ytr), 'val': (Xval, Yval),'test': (Xtest, Ytest),}[split]
    emb = C[x] # (N, block_size, n_embd)
    embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
    hpreact = embcat @ W1 + b1
    hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
    h = torch.tanh(hpreact) # (N, n_hidden)
    logits = h @ W2 + b2 # (N, vocab_size)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.0673816204071045
val 2.1062190532684326


### train 2.0673816204071045
### val 2.1062190532684326

In [ ]:
g = torch.Generator().manual_seed(2147483647 + 10)

for i in range(20):
    out = []
    context =[0]*block_size

    while True:
        emb = C[torch.tensor([context])] # (1,block_size,d)
        embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
        hpreact = embcat @ W1 + b1
        hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
        h = torch.tanh(hpreact) # (N, n_hidden)
        logits = h @ W2 + b2 # (N, vocab_size)

        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        context = context[1:]+[ix]
        out.append(ix)
        if ix==0:
            break

    print(''.join(itos[i] for i in out))

mora.
mayah.
see.
mad.
ryla.
ren.
ruthadraegen.
chedielin.
shi.
jenleigh.
sananaraelyn.
malaia.
noshubergihirael.
kindreth.
konnie.
cayu.
zayven.
jamyleyeh.
yuma.
myston.
